# Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv())
import os   

from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-crewai-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [2]:
from smolagents import DuckDuckGoSearchTool

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [3]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [19]:
#Groq
os.environ["OPENAI_API_KEY"] = os.environ["GROQ_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["GROQ_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = "meta-llama/llama-4-scout-17b-16e-instruct"

In [4]:
#Gemini API
# If you are using Gemini API, set the OpenAI API key to the Gemini API key.
os.environ["OPENAI_API_KEY"] = os.environ["GEMINI_API_KEY"] 
os.environ["OPENAI_MODEL_NAME"] = 'gemini/gemini-1.5-flash'

In [21]:
#Openrouter
os.environ["OPENAI_API_KEY"] = os.environ["OPENROUTER_API_KEY"] 
os.environ["OPENAI_API_BASE"] = os.environ["OPENROUTER_API_BASE"] 
os.environ["OPENAI_MODEL_NAME"] = "mistralai/mistral-small-24b-instruct-2501:free"

## Creating Agents

In [5]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

In [6]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools

In [7]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [8]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [10]:
from crewai.tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [11]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [12]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [14]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [15]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [16]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=True,
	memory=False
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3c1fe0e9-8c03-4576-8a18-e83a086063cf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 1c523c89-35a2-4e63-88ed-d22320d076f4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 1c523c89-35a2-4e63-88ed-d22320d076f4
       Status: Executing Task...
    └── 🤖 Agent: Sales Representative
            Status: In Progress

# Agent: Sales Representative
## Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


🤖 Agent: Sales Representative
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: Lead Sales Representative
## Final Answer: 
**Email 1: Initial Outreach**

Subject: Enhancing DeepLearning.AI's Impact with CrewAI

Dear Dr. Ng,

My name is [Your Name], Lead Sales Representative at CrewAI.  I've been following DeepLearning.AI's incredible success in democratizing AI education, particularly the recent launches of your Generative AI and Prompt Engineering courses.  These initiatives clearly demonstrate your commitment to providing learners with cutting-edge skills.

At CrewAI, we provide AI-powered tools designed to boost learner engagement and streamline course creation. We believe our solutions could significantly enhance the learning experience for your 7 million+ students and optimize your course development workflow.

Would you be open to a brief introductory call next week to explore how CrewAI can support DeepLearning.AI's continued growth?

Sincerely,

[Your Name]
Lead Sales Representative, CrewAI


**Email 2: Following Up**

Subject: Following Up: Cr

🚀 Crew: crew
├── 📋 Task: 1c523c89-35a2-4e63-88ed-d22320d076f4
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 8b09bec2-2692-4069-b3f6-714be92d6a6b
       Status: Executing Task...
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 1c523c89-35a2-4e63-88ed-d22320d076f4
│      Assigned to: Sales Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Sales Representative
│           Status: ✅ Completed
└── 📋 Task: 8b09bec2-2692-4069-b3f6-714be92d6a6b
       Assigned to: Lead Sales Representative
       Status: ✅ Completed
    └── 🤖 Agent: Lead Sales Representative
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8b09bec2-2692-4069-b3f6-714be92d6a6b                                                                     │
│  Agent: Lead Sales Representative                                                                               │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3c1fe0e9-8c03-4576-8a18-e83a086063cf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result.raw)

**Email 1: Initial Outreach**

Subject: Enhancing DeepLearning.AI's Impact with CrewAI

Dear Dr. Ng,

My name is [Your Name], Lead Sales Representative at CrewAI.  I've been following DeepLearning.AI's incredible success in democratizing AI education, particularly the recent launches of your Generative AI and Prompt Engineering courses.  These initiatives clearly demonstrate your commitment to providing learners with cutting-edge skills.

At CrewAI, we provide AI-powered tools designed to boost learner engagement and streamline course creation. We believe our solutions could significantly enhance the learning experience for your 7 million+ students and optimize your course development workflow.

Would you be open to a brief introductory call next week to explore how CrewAI can support DeepLearning.AI's continued growth?

Sincerely,

[Your Name]
Lead Sales Representative, CrewAI


**Email 2: Following Up**

Subject: Following Up: CrewAI and DeepLearning.AI

Dear Dr. Ng,

I hope this email finds you well.

I'm following up on my previous email regarding CrewAI's potential to enhance DeepLearning.AI's educational offerings.  We've helped numerous educational institutions improve learner engagement, retention, and course efficiency, resulting in measurable improvements in learning outcomes.

Our platform offers personalized learning paths, interactive elements, and data-driven insights that can help DeepLearning.AI further refine its curriculum and personalize the learning experience for each student.

Would you be available for a 15-minute call on [Date/Time] or [Date/Time]?

Sincerely,

[Your Name]
Lead Sales Representative, CrewAI


**Email 3:  Pilot Program Proposal**

Subject:  Pilot Program Proposal:  CrewAI and DeepLearning.AI

Dear Dr. Ng,

Following up on our previous conversations, I'd like to propose a pilot program to showcase CrewAI's capabilities.  We could integrate our platform into one of your existing courses, perhaps your popular "Generative AI for Everyone" specialization, to demonstrate the tangible benefits.

This pilot program would allow your team to experience firsthand how CrewAI can:

* Increase student engagement and completion rates.
* Streamline the course creation and update process.
* Provide data-driven insights to inform future curriculum development.

We're confident that the results of this pilot will demonstrate a significant return on investment.  I've attached a more detailed proposal outlining the scope and timeline of the pilot program.  Would you be available for a call to discuss this further?

Sincerely,

[Your Name]
Lead Sales Representative, CrewAI